In [39]:
import sys
print(sys.path)

['C:\\Users\\mgamb\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\mgamb\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\mgamb\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\mgamb\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\mgamb\\DevAcc2024\\.venv', '', 'c:\\Users\\mgamb\\DevAcc2024\\.venv\\Lib\\site-packages', 'c:\\Users\\mgamb\\DevAcc2024\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\mgamb\\DevAcc2024\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\mgamb\\DevAcc2024\\.venv\\Lib\\site-packages\\Pythonwin']


In [40]:
from dotenv import load_dotenv
import os

from openai import OpenAI

import requests

# Load Environment Variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

redirect_id = os.getenv("TWITCH_REDIRECT_URI")
twitch_client_id = os.getenv("TWITCH_CLIENT_ID")
twitch_access_token = os.getenv("TWITCH_TOKEN_URL")

URL = "https://id.twitch.tv/oauth2/token"

# Check if credentials are present
if not twitch_client_id or not twitch_access_token:
    raise ValueError("Twitch credentials not found. Please set TWITCH_CLIENT_ID and TWITCH_ACCESS_TOKEN environment variables.")

In [41]:
# Check if environment variables are loaded
if api_key is None:
    raise ValueError("API key not found. Please set the OPENAI_API_KEY environment variable.")
if twitch_client_id is None or twitch_access_token is None:
    raise ValueError("Twitch credentials not found. Please set the TWITCH_CLIENT_ID and TWITCH_ACCESS_TOKEN environment variables.")

In [42]:
# Set the headers
headers = {
    'Client-ID': twitch_client_id,
    'Authorization': f'Bearer {twitch_access_token}',
}

In [43]:
# Make a request to IGDB API

response = requests.post(
    'https://api.igdb.com/v4/games',
    headers=headers,
    data='fields name,genres.name,platforms.name,release_dates.human; limit 10;'
)

In [44]:
# Print the Response
if response.status_code == 200:
    games_data = response.json()
    for game in games_data:
        print(game)
else:
    print(f"Failed to fetch data: {response.status_code}, {response.text}")

Failed to fetch data: 401, {
  "message": "Authorization Failure. Have you tried:",
  "Tip 1":   "Ensure you are sending Authorization and Client-ID as headers.",
  "Tip 2":   "Ensure Authorization value starts with 'Bearer ', including the space",
  "Tip 3":   "Ensure Authorization value ends with the App Access Token you generated, NOT your Client Secret.",
  "Docs":    "https://api-docs.igdb.com/#authentication",
  "Discord": "https://discord.gg/igdb"
}



Check if API Key is loaded

In [ ]:
if api_key is None:
    raise ValueError("API key not found. Please set the OPENAI_API_KEY environment variable.")

Configure Logging

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

AI Game Analytics 

In [ ]:
assistant = client.beta.assistants.create(
    name="Video Game Expert",
    instructions="""
    You are an AI assistant specializing in video games. You can provide detailed analytics and insights into gameplay, helping players track their progress and identify areas for improvement. 
    You can answer questions about game completion times, strategies to progress past difficult sections, fastest speedrun times, and general tips and tricks.
    For example:
    - "How long would it take to fully complete Super Mario Bros. on NES?"
    - "How do I progress past the Water Temple in The Legend of Zelda: Ocarina of Time?"
    - "What is the fastest speedrun time for Sonic the Hedgehog?"
    - "Give me tips and tricks to improve my gameplay in Fortnite."
    Use up-to-date information and provide the best possible answers to enhance the user's gaming experience.
    """,
    tools=[{"type": "code_interpreter"}],  # Add more tools if needed
    model="gpt-4o"
)

Create a Thred

In [ ]:
from openai import APIError

def create_thread():
    try:
        thread = client.beta.threads.create()
        logging.info(f"Thread created: {thread}")
        return thread
    except APIError as e:
        logging.error(f"Error creating thread: {e}")
        return None
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return None

Add a Message to a Thread

In [ ]:
def add_message_to_thread(thread_id, message_content):
    try:
        message = client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=message_content
        )
        logging.info(f"Message added to thread: {message}")
        return message
    except APIError as e:
        logging.error(f"Error adding message to thread: {e}")
        return None
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return None

Run the Assistant

In [ ]:
def run_assistant(thread_id, assistant_id):
    try:
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id
        )
        logging.info(f"Run created: {run}")
        return run
    except APIError as e:
        logging.error(f"Error running assistant: {e}")
        return None
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        return None

Retreive and Display the Assistant's Response

In [ ]:
import time

def display_assistant_response(thread_id, run_id):
    try:
        # Add a delay to allow the assistant to process
        time.sleep(5)  # Increase the delay to ensure processing time

        messages = client.beta.threads.messages.list(thread_id=thread_id)
        for message in reversed(messages.data):
            if message.role == "assistant":
                for content in message.content:
                    print(f"Assistant: {content.text.value}")
                return
        logging.info("No response from assistant.")
    except APIError as e:
        logging.error(f"Error retrieving messages: {e}")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")

Example:

In [ ]:
thread = create_thread()
if thread:
    user_input = input("Enter your gameplay data or question for analysis: ")
    message = add_message_to_thread(thread.id, user_input)
    if message:
        run = run_assistant(thread.id, assistant.id)
        if run:
            display_assistant_response(thread.id, run.id)